In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
from pycaret.classification import *
from src.utils import *
from src.calcEMA import *
from src.myenv import *
import plotly.express as px


In [2]:
# Variables
# ETCUSDT BTCUSDT
# symbol = 'ETHUSDT'
symbol = 'BTCUSDT'
# lightgbm  xgboost lr et rf
estimator = 'et'
_compare_models = True

start_train_date = '2010-01-01'  # train < and test >=
start_test_date = '2023-01-01'  # train < and test >=

stop_loss = 2.0
label = 'status'
regression_times = 0# 24 * 30 * 2  # horas
times_regression_profit_and_loss = 24
# numeric_features=['open', 'high', 'low', 'volume', 'close', 'rsi']
numeric_features = ['open', 'high', 'low', 'volume', 'close', ]
_calc_rsi = True
nomalize = True

### Metadata

<code>
Field Name - Description</br>
open_time - Kline Open time in unix time format</br>
open - Open Price</br>
high - High Price</br>
low	- Low Price</br>
close	- Close Price</br>
volume - Volume</br>
close_time - Kline Close time in unix time format</br>
quote_volume - Quote Asset Volume</br>
count	- Number of Trades</br>
taker_buy_volume - Taker buy base asset volume during this period</br>
taker_buy_quote_volume - Taker buy quote asset volume during this period</br>
ignore - Ignore</br>
</code>

In [3]:
use_cols = date_features + float_kline_cols + integer_kline_cols
use_cols.remove('ignore')
print(use_cols)
all_data = read_data(f'{datadir}/{symbol}', all_cols=None, use_cols=use_cols)
all_data = parse_type_fields(all_data)

# Filter all data
all_data = all_data[(all_data['open_time'] >= start_train_date)].copy()

print(all_data.info())

['open_time', 'open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'number_of_trades']
read_data: Start reading file:  /home/marcelo/des/mg_crypto_trader/src/data/BTCUSDT/BTCUSDT.csv
<class 'pandas.core.frame.DataFrame'>
Int64Index: 53052 entries, 0 to 53051
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     53052 non-null  datetime64[ns]
 1   open                          53052 non-null  float32       
 2   high                          53052 non-null  float32       
 3   low                           53052 non-null  float32       
 4   close                         53052 non-null  float32       
 5   volume                        53052 non-null  float32       
 6   quote_asset_volume            53052 non-null  float32       
 7   number_of_trades              53052

In [4]:
if _calc_rsi:
  all_data = calc_RSI(all_data)
  numeric_features.append('rsi')
  all_data.dropna(inplace=True)
print(all_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53038 entries, 14 to 53051
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     53038 non-null  datetime64[ns]
 1   open                          53038 non-null  float32       
 2   high                          53038 non-null  float32       
 3   low                           53038 non-null  float32       
 4   close                         53038 non-null  float32       
 5   volume                        53038 non-null  float32       
 6   quote_asset_volume            53038 non-null  float32       
 7   number_of_trades              53038 non-null  int16         
 8   taker_buy_base_asset_volume   53038 non-null  float32       
 9   taker_buy_quote_asset_volume  53038 non-null  float32       
 10  rsi                           53038 non-null  float32       
dtypes: datetime64[ns](1), float

In [5]:
all_cols = date_features + numeric_features
print('All Columns: ', all_cols)
all_data[all_cols]
#all_data = all_data[[all_cols]].copy()

All Columns:  ['open_time', 'open', 'high', 'low', 'volume', 'close', 'rsi']


,open_time,open,high,low,volume,close,rsi
14,2017-08-17 18:00:00,4289.240234,4302.450195,4218.680176,51.275162,4256.970215,44.570000
15,2017-08-17 19:00:00,4241.910156,4335.299805,4200.740234,46.710850,4325.229980,51.959999
16,2017-08-17 20:00:00,4307.560059,4354.839844,4258.560059,48.975471,4346.740234,54.049999
17,2017-08-17 21:00:00,4346.740234,4369.689941,4309.229980,61.773037,4333.549805,52.540001
18,2017-08-17 22:00:00,4333.549805,4359.129883,4310.000000,57.376141,4336.799805,52.889999
...,...,...,...,...,...,...,...
53047,2023-09-10 19:00:00,25728.230469,25860.000000,25570.570312,1903.835815,25800.000000,47.320000
53048,2023-09-10 20:00:00,25799.990234,25911.000000,25771.300781,2201.600098,25820.349609,50.259998
53049,2023-09-10 21:00:00,25820.349609,25908.949219,25815.500000,763.040100,25902.810547,60.009998
53050,2023-09-10 22:00:00,25902.820312,26033.660156,25851.849609,1520.061523,25852.339844,53.150002


In [6]:
all_data = regression_PnL(all_data, label, stop_loss, times_regression_profit_and_loss)
all_data

,open_time,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,status
14,2017-08-17 18:00:00,4289.240234,4302.450195,4218.680176,4256.970215,51.275162,2.182594e+05,194,45.031239,1.916552e+05,44.570000,CAI_2.0
15,2017-08-17 19:00:00,4241.910156,4335.299805,4200.740234,4325.229980,46.710850,1.995219e+05,202,42.010857,1.794710e+05,51.959999,CAI_2.0
16,2017-08-17 20:00:00,4307.560059,4354.839844,4258.560059,4346.740234,48.975471,2.105489e+05,247,42.883137,1.843113e+05,54.049999,CAI_2.0
17,2017-08-17 21:00:00,4346.740234,4369.689941,4309.229980,4333.549805,61.773037,2.682849e+05,248,51.152618,2.221201e+05,52.540001,CAI_2.0
18,2017-08-17 22:00:00,4333.549805,4359.129883,4310.000000,4336.799805,57.376141,2.483683e+05,236,44.313545,1.918982e+05,52.889999,CAI_2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
53047,2023-09-10 19:00:00,25728.230469,25860.000000,25570.570312,25800.000000,1903.835815,4.893582e+07,-30842,922.614929,2.371798e+07,47.320000,ESTAVEL
53048,2023-09-10 20:00:00,25799.990234,25911.000000,25771.300781,25820.349609,2201.600098,5.689209e+07,-31784,1185.309448,3.062813e+07,50.259998,ESTAVEL
53049,2023-09-10 21:00:00,25820.349609,25908.949219,25815.500000,25902.810547,763.040100,1.974176e+07,18871,416.213409,1.076734e+07,60.009998,ESTAVEL
53050,2023-09-10 22:00:00,25902.820312,26033.660156,25851.849609,25852.339844,1520.061523,3.940769e+07,29567,761.259705,1.973595e+07,53.150002,ESTAVEL


In [7]:
if regression_times > 0:
    for nf in numeric_features.copy():
        for i in range(1, regression_times + 1):
            col = nf + "_" + str(i)
            all_data[col] = all_data[nf].shift(i)
            numeric_features.append(col)

    all_data.dropna(inplace=True)
    all_data.info()

In [8]:
train_data = all_data[(all_data['open_time'] >= start_train_date) & (all_data['open_time'] < start_test_date)]
train_data = train_data.sort_values(date_features)
train_data

,open_time,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,status
14,2017-08-17 18:00:00,4289.240234,4302.450195,4218.680176,4256.970215,51.275162,2.182594e+05,194,45.031239,1.916552e+05,44.570000,CAI_2.0
15,2017-08-17 19:00:00,4241.910156,4335.299805,4200.740234,4325.229980,46.710850,1.995219e+05,202,42.010857,1.794710e+05,51.959999,CAI_2.0
16,2017-08-17 20:00:00,4307.560059,4354.839844,4258.560059,4346.740234,48.975471,2.105489e+05,247,42.883137,1.843113e+05,54.049999,CAI_2.0
17,2017-08-17 21:00:00,4346.740234,4369.689941,4309.229980,4333.549805,61.773037,2.682849e+05,248,51.152618,2.221201e+05,52.540001,CAI_2.0
18,2017-08-17 22:00:00,4333.549805,4359.129883,4310.000000,4336.799805,57.376141,2.483683e+05,236,44.313545,1.918982e+05,52.889999,CAI_2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
46976,2022-12-31 19:00:00,16577.779297,16590.060547,16565.099609,16570.140625,4044.433594,6.704605e+07,-14485,2067.921631,3.428087e+07,50.099998,ESTAVEL
46977,2022-12-31 20:00:00,16570.140625,16574.970703,16564.089844,16568.599609,2622.143555,4.344849e+07,25169,1311.381104,2.172951e+07,49.709999,ESTAVEL
46978,2022-12-31 21:00:00,16568.189453,16571.640625,16544.119141,16548.279297,3618.773926,5.992803e+07,-26007,1746.413330,2.892190e+07,44.779999,ESTAVEL
46979,2022-12-31 22:00:00,16548.279297,16567.490234,16470.000000,16520.810547,6695.136230,1.106669e+08,22861,3119.958252,5.157153e+07,39.130001,ESTAVEL


In [9]:
setup = setup(train_data,
              train_size=0.7,
              target=label,
              numeric_features=numeric_features,
              date_features=['open_time'],
              create_date_columns=["hour", "day", "month"],
              fold_strategy='timeseries',
              fold=3,
              session_id=123,
              normalize=nomalize,
              use_gpu=False,
              verbose=True,
              n_jobs=20,
              )

,Description,Value
0,Session id,123
1,Target,status
2,Target type,Multiclass
3,Target mapping,"CAI_2.0: 0, ESTAVEL: 1, SOBE_2.0: 2"
4,Original data shape,"(46967, 12)"
5,Transformed data shape,"(46967, 14)"
6,Transformed train set shape,"(32876, 14)"
7,Transformed test set shape,"(14091, 14)"
8,Numeric features,6
9,Date features,1


In [10]:
#Accuracy	AUC	Recall	Prec.	F1	Kappa	MCC
#model_name = 'lightgbm' 'xgboost'
if _compare_models:
  best = setup.compare_models()
  estimator = setup.pull().index[0]
  print('Estimator: ' + estimator)
else:
  best = setup.create_model(estimator)

best

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.7391,0.8894,0.7391,0.7394,0.7382,0.6062,0.6070,0.3067
rf,Random Forest Classifier,0.6926,0.8569,0.6926,0.6926,0.6910,0.5357,0.5369,0.7000
dt,Decision Tree Classifier,0.6497,0.7363,0.6497,0.6495,0.6495,0.4725,0.4725,0.5333
lightgbm,Light Gradient Boosting Machine,0.6318,0.8098,0.6318,0.6313,0.6284,0.4433,0.4455,530.5467
xgboost,Extreme Gradient Boosting,0.6315,0.8099,0.6315,0.6303,0.6286,0.4433,0.4449,200.3067
catboost,CatBoost Classifier,0.6262,0.8031,0.6262,0.6250,0.6232,0.4354,0.4370,1.3667
gbc,Gradient Boosting Classifier,0.5359,0.7218,0.5359,0.5353,0.5155,0.2926,0.3031,6.4333
knn,K Neighbors Classifier,0.5297,0.7103,0.5297,0.5294,0.5229,0.2865,0.2900,0.6800
ada,Ada Boost Classifier,0.4692,0.6486,0.4692,0.4614,0.4350,0.1886,0.2003,0.5600
lda,Linear Discriminant Analysis,0.4509,0.6218,0.4509,0.4344,0.3888,0.1633,0.1802,0.0967


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

Estimator: et


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=100, n_jobs=20, oob_score=False,
                     random_state=123, verbose=0, warm_start=False)

In [11]:
best = tune_model(best)
best

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5201,0.7032,0.5201,0.5228,0.5145,0.2803,0.2844
1,0.5231,0.7073,0.5231,0.5268,0.5158,0.2864,0.2918
2,0.5321,0.7097,0.5321,0.5346,0.5265,0.2987,0.3029
Mean,0.5251,0.7067,0.5251,0.5281,0.5189,0.2885,0.2930
Std,0.0051,0.0027,0.0051,0.0049,0.0054,0.0077,0.0076


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=100, n_jobs=20, oob_score=False,
                     random_state=123, verbose=0, warm_start=False)

In [12]:
test_data = all_data[all_data['open_time'] >= start_test_date]
test_data = test_data.sort_values(date_features)

In [13]:
# predict on test set
holdout_pred = predict_model(best)
print(holdout_pred['prediction_score'].mean())

holdout_pred[[label, 'prediction_score']].groupby(label).mean()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.8184,0.9441,0.8184,0.8192,0.8180,0.7260,0.7268


0.6913412816691504


,prediction_score
status,
0,0.689677
1,0.722942
2,0.660875


In [14]:
predict = predict_model(best, data=test_data.drop(columns=[label]))
predict[label] = test_data[label]
predict['_score'] = predict['prediction_label'] == predict[label]
print('Score Mean:', predict['_score'].mean())

Score Mean: 0.18596606819304892


In [15]:
predict[[label, '_score']].groupby(label).mean()

,_score
status,
CAI_2.0,0.623561
ESTAVEL,0.032733
SOBE_2.0,0.239028


In [16]:
final_predict = finalize_model(best) # data=test_data.sort_values(date_features).drop(columns=[label]))
final_predict

Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('date_feature_extractor',
                 TransformerWrapper(exclude=None, include=['open_time'],
                                    transformer=ExtractDateTimeFeatures(features=['hour',
                                                                                  'day',
                                                                                  'month']))),
                ('numerical_imputer',
                 TransformerWrapper(...
                 ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0,
                                      class_weight=None, criterion='gini',
                                      max_depth=None, max_features='sqrt',
                                      max_leaf_nodes=None, max_samples=None,
                                      min_impurity_decrease=0.0,
                                      min_samples_leaf=1, min_samples_split=2,
                                      min_weight_fraction_leaf=0.0,
                                      n_estimators=100, n_jobs=20,
                                      oob_score=False, random_state=123,
                                      verbose=0, warm_start=False))],
         verbose=False)

In [17]:
_predict = predict_model(final_predict, data=test_data.sort_values(date_features).drop(columns=[label]))
_predict[label] = test_data[label]
_predict['_score'] = _predict['prediction_label'] == _predict[label]
print('Score Mean:', _predict['_score'].mean())

Score Mean: 0.18810739581617525


In [18]:
_predict[[label, '_score']].groupby(label).mean()

,_score
status,
CAI_2.0,0.567759
ESTAVEL,0.033552
SOBE_2.0,0.296238


In [19]:
if True:
  for i in range(1, 9999):
    filename = f'{symbol}_{estimator}_SL_{stop_loss}_RT_{regression_times}_RPL_{times_regression_profit_and_loss}_{i}'
    print('Model file name: ', filename + '.pkl')
    if os.path.exists(filename + '.pkl'):
      continue
    else:    
      setup.save_model(final_predict, filename)
      break

Model file name:  BTCUSDT_et_SL_2.0_RT_0_RPL_24_1.pkl
Transformation Pipeline and Model Successfully Saved
